# Homework 3: Car Prices

## Cleaning and Exploring Data with `pandas`; Applying Simple Linear Regression

## Due Date: Friday, Oct 6, 11:59 PM
You must submit this assignment to Gradescope by the on-time deadline, Friday, Oct 6, 11:59 PM. Loading in additional cells is permitted. **You may ask questions about the homework with your peers, though you are required to submit your own, individual work.**

*Distributed on September 29th, 2023.*

Hosted and maintained by [SAAS Berkeley](https://saas.berkeley.edu)

In [ ]:
#just run this cell

import warnings
warnings.filterwarnings('ignore')

#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

### Abstract

An automotive company from China, [Geely Auto](https://global.geely.com/), has plans to establish its presence in the United States market. The company intends to set up a local manufacturing unit and produce cars to rival U.S. and European competitors.

To reach this goal, Geely Auto has enlisted the services of a car consultancy firm to delve into the various elements that influence car pricing. Specifically, the focus is on discerning the dynamics affecting car prices in the American market, as these are likely to differ substantially from those in the Chinese market. The key tasks the company seeks to address are:

- **Identifying the variables that play a role in determining a car's price.**


- **Evaluating the extent to which these identified variables can accurately depict a car's price.**


In an attempt address these queries, the consultancy firm has compiled a comprehensive dataset from multiple market surveys, encompassing diverse car models across the American market.

### Business Objective (your task as data scientists)
Your task involves developing a model to project car prices using the provided independent variables. More specifically, you will be looking and analyzing the collected data to decipher the correlations between prices and the various features impacting the price of U.S. and European-manufactured vehicles. This homework assignment will dive deeply into the basic of Exploratory Data Analysis and its real-world applications.

The consultant had given you a csv file named `CarPrices`. Let's start with the following steps:

- Importing data using the pandas library
- Understanding the structure of the data

In [ ]:
cars = pd.read_csv("CarPrices.csv")

In [ ]:
#just run this cell
cars.head()

In [ ]:
#just run this cell
cars.shape

In [ ]:
cars.info()

In [ ]:
cars.describe()

Examining the dataset, the `VehicleName` column seems to contain strings that combine the make (or company name/brand) of the car and the model name, separated by a space.

We could leave the dataframe like this, but we could further add modifications. `VehicleName`amalgamates two distinct pieces of information: the car's make (or brand) and its model name. Isolating the brand information in a new `CompanyName` column could be useful for analysis, grouping, or comparing cars by their manufacturers. Let's start with this first.

In [ ]:
#extract company name and add it as a new column
cars = cars.assign(CompanyName=cars['VehicleName'].str.split(' ').str[0])
#drop the original CarName column
cars.drop(['VehicleName'], axis=1, inplace=True)

cars.head()

Now we have abstracted the model of the car are are left with the manufacturer.

### Question 1

Before we proceed, let's check to see if there are any duplicate entries in our dataframe

**Exercise 1:** In the cell below, write a line of code which to check whether or not there are duplicate entries. Then, set `are_duplicates` to either `True` if there were duplicate entries in the dataframe, or `False` otherwise.

**Hint:** There is a pandas function which checks for duplicates. (read the pandas documentation)

In [ ]:
#check whether or not there are duplicate entries in car
...

In [ ]:
are_duplicates = ...

During lecture, we saw an example of a boxplot. It’s a standardized way of displaying the distribution of data based on the minimum, first quartile (Q1), median, third quartile (Q3), and maximum.

### Question 2

**Exercise 2a:** Plot the boxplot distribution and spread of vehicle prices below using the `VehiclePrice` column.

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Vehicle Price Spread')
...

plt.show()

In [ ]:
cars.VehiclePrice.describe(percentiles = [0.25,0.50,0.75,0.85,0.90,1])

**Exercise 2b:** Is this distribution right-skewed, left-skewed, or symmetric? What other inferences can you make from this visualization? 

*your response here*

Let's now analyze the frequency distribution of the two categorical variables `CompanyName` and `BodyStlye`.

In [ ]:
#just run this cell
fig, axes = plt.subplots(1, 2, figsize=(25, 6))

sns.countplot(x='CompanyName', data=cars, ax=axes[0])
axes[0].set_title('Companies Histogram')
axes[0].set_xlabel('Car company')
axes[0].set_ylabel('Frequency of company')
axes[0].tick_params(axis='x', rotation=45) 

sns.countplot(x='BodyStyle', data=cars, ax=axes[1])
axes[1].set_title('Car Type Histogram')
axes[1].set_xlabel('Car Type')
axes[1].set_ylabel('Frequency of Car type')
axes[1].tick_params(axis='x', rotation=45)  

plt.tight_layout()
plt.show()

**Exercise 2c:** What inferences can you make from these visualizations? 

*your response here*

In the context of cars, symboling is related to the insurance risk rating of a vehicle. It is a value given by insurance companies: a larger value indicates a greater risk, implying higher insurance premiums. You'll typically see sportier, high-powered cars with a larger symboling, reflecting their associated greater risk and, consequently, higher insurance premiums, whereas safer, more family-oriented cars may have lower symboling ratings.

In [ ]:
#just run this cell
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title('Symboling Histogram')
sns.countplot(x = cars.Symboling, palette=("cubehelix"))

plt.subplot(1,2,2)
plt.title('Symboling vs Price')
sns.boxplot(x=cars.Symboling, y=cars.VehiclePrice, palette=("cubehelix"))

plt.show()

**Exercise 2d:** What inferences can you make from these visualizations? 

*your response here*

Now, lastly, let's take a look at the `CompanyName` and `BodyType` to see how this may influence the price of a car. 

*Note:* We didn't explicitly discuss the `groupby` method in lecture, but it is a very important method in practice for aggregating data together. I found [this video](https://www.youtube.com/watch?v=2I2E1ZbF8pg&ab_channel=NoureddinSadawi) to be very helpful in further explaining this process.

In [ ]:
#just run this cell
plt.figure(figsize=(25, 6))

df = pd.DataFrame(cars.groupby(['CompanyName'])['VehiclePrice'].mean().sort_values(ascending = False))
df.plot.bar()
plt.title('Company Name vs Average Price')
plt.show()

df = pd.DataFrame(cars.groupby(['BodyStyle'])['VehiclePrice'].mean().sort_values(ascending = False))
df.plot.bar()
plt.title('Car Type vs Average Price')
plt.show()

**Exercise 2e:** Which car manufacturers and car types on average were priced the highest?

*your response here*

### Question 3

As you can see, this dataset is incredibly diverse with multiple different features we can use to make a prediction on car prices. We call this *multi-dimensional*. 

Now, relying on a single feature *might* not capture the comprehensive pricing dynamics, leading to potential loss of information and predictive accuracy. A multivariate approach incorporating multiple features would be more appropriate to capture the relationships and dependencies between different features and the car price. Don't worry! We will be exploring this in the future, but for now, we will assume one feature to suffice for the task at hand.

Based on the EDA, you were to get a sense as to how certain features may impact the price of a car. With your newfound knowledge, you are now asked to select a feature that appears to have a linear relationship with `VehiclePrice` for the Simple Linear Regression model. **However you select this feature is up to you.** In the cell below:

- fill in the blanks with your respective feature and apply the model to derive a regression line for prediction
- print the equation for the regression line
- plot the regression line based on the selected feature using a scatter plot

*Hints:*

- Take into account the form of quantitative data you're using as a feature (discrete vs continuous). you might need to look at other features in the dataset, features we didn't explicitly perform EDA on above. What form of data will be better here?
- This process is outline in the lecture notebook

In [ ]:
#we initialize the Linear Regression model
lm = linear_model.LinearRegression()

lm.fit(...)

#output the regression line equation
print("Our Regression Line is Y = " ...)

#plotting the selected feature
plt.scatter(x = ..., y = ...)
plt.xlabel('Engine Size')
plt.ylabel('Car Price')

#calculate and plot the REGRESSION LINE (not the selected feature)
x = np.linspace(min(cars[...]), max(cars[...]), 100) #fill the "..." with the feature you selected
plt.plot(...)

plt.show()

### Question 4

As a last task, let's now assess the residuals of your linear model to understand how well your selected feature explains the variability in car prices. Fill in the following code with your selected feature:

In [ ]:
# Plotting the residual plot
sns.residplot(x = cars[...], y = cars["VehiclePrice"])

plt.xlabel(...)
plt.ylabel('Residuals')
plt.show()

**Exercise 4a:** What inferences can you make about the residual plot shown above. Was your selected feature *good* at making predictions for car prices?

*your response here*

### Congratulations! You finished the homework assignment!

You finished the homework assignment! 

To summarize, we practiced and applied SLR to real-world data, reinforcing our understanding of the model and gaining insights into its implications. While SLR provided us with valuable insights and a solid foundation in regression modeling, hopefully this assignment showed you that it might not be universally applicable or the most effective in all scenarios, especially when dealing with multifaceted and intricate relationships within the data.

Next week, we will dive deeper into more sophisticated and nuanced modeling techniques, explore regularization methods to enhance model generalization and prevent overfitting, and delve into concepts of bias and variance to analytically assess our models and understand the trade-offs involved.

**Download the jupyter notebook by navigating to File>Save and Export Notebook As>PDF. Then, navigate to Gradescope and submit the file.**

**If you don't have the dependencies necessary to compile a notebook to PDF, follow the instructions outlined by jupyter.**

**PM Carlos if you are having trouble with this step. You may also just print the page (i.e. ctrl+P), download as a PDF, and submit from there.**